In [1]:
from lusidtools.jupyter_tools import toggle_code

"""Portfolio Entitlements

Demonstrates how to create policies/access control to various date items.

Attributes
----------
entitlements
"""

toggle_code("Toggle Docstring")

# Portfolio Entitlements

## 1. Setup

In [3]:
# Import Libraries
import os
import json
import pytz

from datetime import datetime, timedelta

import finbourne_access
import finbourne_identity
import lusid
import lusid.models as lm
import pandas as pd
from lusid import ApiException
from datetime import date

from finbourne_access import models as access_models
from finbourne_identity import models as identity_models
from lusidjam import RefreshingToken
from IPython.core.display import HTML

# Set DataFrame display formats
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)
pd.options.display.float_format = "{:,.2f}".format
display(HTML("<style>.container { width:90% !important; }</style>"))

# Authenticate our user and create our API client
# secrets_path = os.getenv("FBN_SECRETS_PATH")
secrets_path = '/Users/msingh/Projects/lusidws/secrets.json'

lusid_api_factory = lusid.utilities.ApiClientFactory(
    token=RefreshingToken(),
    api_secrets_filename=secrets_path,
    app_name="LusidJupyterNotebook",
)

api_client = lusid_api_factory.api_client

lusid_api_url = api_client.configuration.host
access_api_url = lusid_api_url[: lusid_api_url.rfind("/") + 1] + "access"
identity_api_url = lusid_api_url[: lusid_api_url.rfind("/") + 1] + "identity"

access_api_factory = finbourne_access.utilities.ApiClientFactory(
    token=api_client.configuration.access_token,
    access_url=access_api_url,
    app_name="LusidJupyterNotebook",
)

identity_api_factory = finbourne_identity.utilities.ApiClientFactory(
    token=api_client.configuration.access_token,
    api_url=identity_api_url,
    app_name="LusidJupyterNotebook",
)

api_status = pd.DataFrame(
    lusid_api_factory.build(lusid.api.ApplicationMetadataApi)
    .get_lusid_versions()
    .to_dict()
)

display(api_status)

,api_version,build_version,excel_version,links
0,v0,0.6.8392.0,0.5.2556,"{'relation': 'RequestLogs', 'href': 'http://bl..."


In [5]:
policies_api = access_api_factory.build(finbourne_access.PoliciesApi)
portfolios_api = lusid_api_factory.build(lusid.PortfoliosApi)
transactions_api = lusid_api_factory.build(lusid.TransactionPortfoliosApi)
access_roles_api = access_api_factory.build(finbourne_access.RolesApi)
identity_roles_api = identity_api_factory.build(finbourne_identity.RolesApi)
users_api = identity_api_factory.build(finbourne_identity.UsersApi)

## 1.2 Create Portfolios

In [7]:
def check_portfolio(code, scope):
    try:
        result = portfolios_api.delete_portfolio(code=code, scope=scope)
        return 'Portfolio successfull deleted'
    except ApiException as e:
        return 'Portfolio does not yet exist'

In [13]:
def create_portfolio(portfolio_code, scope='', display_name='', base_currency='GBP'):
    check_portfolio(portfolio_code, scope)
    request = lm.CreateTransactionPortfolioRequest(
        display_name=display_name,
        code=portfolio_code,
        base_currency=base_currency,
        created=str(date(2020,1,1)))
    try:
        result = transactions_api.create_portfolio(
            scope=scope,
            create_transaction_portfolio_request=request
        )
        return result
    except ApiException as e:
        display(json.loads(e.body)['title'])

In [66]:
PS_FUND_MANAGER = 'Portfolio/PS/FundManager'
FIODE_FUND_MANAGER = 'Portfolio/FIODE/FundManager'
CRIMS_FUND_MANAGER = 'Portfolio/CRIMS/FundManager'

scope = 'prod'
portfolio_code = '1354'
display_name = 'Portfolio ' + portfolio_code
transaction_date = datetime.now()

In [68]:
properties = {
    PS_FUND_MANAGER: lusid.ModelProperty(key=PS_FUND_MANAGER, value=lusid.PropertyValue('PS Singh'))
}

request = lm.CreateTransactionPortfolioRequest(
    display_name=display_name,
    code=portfolio_code,
    base_currency='GBP',
    created=str(date(2021, 12, 24)),
    properties=properties
)

try:
    result = transactions_api.create_portfolio(
            scope=scope,
            create_transaction_portfolio_request=request
        )
    print(result)
except ApiException as e:
    display(json.loads(e.body)['title'])

"Could not create a portfolio with id '1354' because it already exists in scope 'prod'."

In [69]:
properties = {
    FIODE_FUND_MANAGER: lusid.ModelProperty(key=FIODE_FUND_MANAGER, value=lusid.PropertyValue('FIODE Singh'))
}

request = lm.CreateTransactionPortfolioRequest(
    display_name=display_name,
    code=portfolio_code,
    base_currency='GBP',
    created=str(date(2021, 12, 24)),
    properties=properties
)

try:
    result = transactions_api.upsert_portfolio_details(
            scope=scope,
            code=portfolio_code,
            create_portfolio_details=request,
        )
    print(result)
except ApiException as e:
    display(json.loads(e.body)['title'])

{'base_currency': 'GBP',
 'corporate_action_source_id': None,
 'href': 'https://bleugrain.lusid.com/api/api/transactionportfolios/prod/1354/details?effectiveAt=2021-12-24T00%3A00%3A00.0000000%2B00%3A00&asAt=2021-12-24T10%3A54%3A32.8179100%2B00%3A00',
 'instrument_scopes': [],
 'links': [{'description': None,
            'href': 'https://bleugrain.lusid.com/api/api/portfolios/prod/1354?effectiveAt=2021-12-24T00%3A00%3A00.0000000%2B00%3A00&asAt=2021-12-24T10%3A54%3A32.8179100%2B00%3A00',
            'method': 'GET',
            'relation': 'Root'},
           {'description': 'A link to the LUSID Insights website showing all '
                           'logs related to this request',
            'href': 'http://bleugrain.lusid.com/app/insights/logs/0HME64GDPR7LP:00000006',
            'method': 'GET',
            'relation': 'RequestLogs'}],
 'origin_portfolio_id': {'code': '1354', 'scope': 'prod'},
 'sub_holding_keys': [],
 'version': {'as_at_date': datetime.datetime(2021, 12, 24, 10, 5

In [70]:
properties = {
    PS_FUND_MANAGER: lusid.ModelProperty(key=PS_FUND_MANAGER, value=lusid.PropertyValue('PS Singh')),
    FIODE_FUND_MANAGER: lusid.ModelProperty(key=FIODE_FUND_MANAGER, value=lusid.PropertyValue('FIODE Singh')),
    CRIMS_FUND_MANAGER: lusid.ModelProperty(key=CRIMS_FUND_MANAGER, value=lusid.PropertyValue('CRIMS Singh')),
}

request = lm.CreateTransactionPortfolioRequest(
    display_name=display_name,
    code=portfolio_code,
    base_currency='GBP',
    created=str(date(2021, 12, 24)),
    properties=properties
)

try:
    result = transactions_api.upsert_portfolio_details(
            scope=scope,
            code=portfolio_code,
            create_portfolio_details=request
        )
except ApiException as e:
    display(json.loads(e.body)['title'])

In [71]:
request = lm.TransactionRequest(
    transaction_id='TX000000000001',
    type='Buy',
    instrument_identifiers={
        'Figi': 'BBG000NP5GW2'
    },
    transaction_date=transaction_date.isoformat(),
    settlement_date=transaction_date.isoformat(),
    units=3000,
    transaction_price=lm.TransactionPrice(432.32, 'Price'),
    total_consideration=lm.CurrencyAndAmount(3000*432.32, 'GBP')
)

response = transactions_api.upsert_transactions(scope, 
                                                portfolio_code, 
                                                transaction_request=[request])
response = response.to_dict()
display({'response link':response['links'][2]['href']})

ApiException: (400)
Reason: Bad Request
HTTP response headers: HTTPHeaderDict({'Date': 'Fri, 24 Dec 2021 10:54:37 GMT', 'Content-Type': 'application/problem+json; charset=utf-8', 'Content-Length': '641', 'Connection': 'keep-alive', 'X-Rate-Limit-Limit': '1m', 'X-Rate-Limit-Remaining': '4995', 'X-Rate-Limit-Reset': '2021-12-24T10:55:16.3375593Z', 'lusid-meta-success': 'False', 'lusid-meta-requestId': '0HME5DH33K5N8:00000004', 'lusid-meta-correlationId': '0HME5DH33K5N8:00000004', 'lusid-meta-duration': '30', 'Strict-Transport-Security': 'max-age=15724800; includeSubDomains', 'Server': 'FINBOURNE', 'Content-Security-Policy': "default-src 'self' https://*.lusid.com https://*.finbourne.com; script-src 'unsafe-inline' 'self' https://*.lusid.com https://*.finbourne.com; font-src 'self' fonts.googleapis.com; img-src data: 'self' https://*.lusid.com https://*.finbourne.com; style-src 'unsafe-inline' 'self' https://*.lusid.com https://*.finbourne.com; report-uri https://lusid.report-uri.com/r/d/

In [64]:
response = portfolios_api.get_portfolio(scope=scope, code='1351', property_keys=[PS_FUND_MANAGER, FIODE_FUND_MANAGER, CRIMS_FUND_MANAGER])
#print(response)

In [23]:
def upsert_fx(scope, port_code, transaction_date=datetime(2021, 5, 24, tzinfo=pytz.utc)):
    T_id = 'Trasaction_1'
    request = lm.TransactionRequest(
        transaction_id=T_id,
        type='FwdFxBuy',
        instrument_identifiers={
            'Instrument/default/Currency': 'GBP'
        },
        transaction_date=transaction_date.isoformat(),
        settlement_date=transaction_date.isoformat(),
        units=1000,
        transaction_price=lm.TransactionPrice(1, 'Price'),
        total_consideration=lm.CurrencyAndAmount(1000, 'GBP')
    )
    
    response = transactions_api.upsert_transactions(scope, 
                                                    port_code, 
                                                    transaction_request=[request])
    response = response.to_dict()
    display({'response link':response['links'][2]['href']})

In [24]:
port_1250_response = create_portfolio('1250', scope='Source1', display_name='Port 1250', base_currency='GBP')
port_1232_response = create_portfolio('1232', scope='Source2', display_name='Port 1232', base_currency='GBP')
port_6305_response = create_portfolio('6305', scope='Source3', display_name='Port 6305', base_currency='GBP')

In [27]:
upsert_fx('Source1', '1250')
upsert_fx('Source3', '6305', (datetime.now(tz=pytz.utc) + timedelta(days=-15)))

{'response link': 'http://bleugrain.lusid.com/app/insights/logs/0HME5DH33K4DA:00000077'}

{'response link': 'http://bleugrain.lusid.com/app/insights/logs/0HME5DH33K4DA:00000078'}

In [ ]:
lm.CreateTransactionPortfolioRequest(
    
)

## 2. Policy and User creation

### 2.1 Policy Creation

In [ ]:
when_spec = access_models.WhenSpec(
    activate=datetime.now(tz=pytz.utc) + timedelta(day=2),
    deactivate=datetime(2022, 4, 30, tzinfo=pytz.utc)
)

for_spec = access_models.ForSpec(
    effective_date_relative=access_models.EffectiveDateRelative(
        date='Now',
        adjustment=(datetime.now(tz=pytz.utc) + timedelta(days=0)).strftime('-%j'),
        unit='Day',
        relative_to_date_time='BeforeOrOn'
    )
)

In [18]:
(datetime.now(tz=pytz.utc) + timedelta(days=-15))

datetime.datetime(2021, 12, 9, 8, 27, 28, 507099, tzinfo=<UTC>)

In [29]:
(datetime.now(tz=pytz.utc) + timedelta(days=0))

datetime.datetime(2021, 12, 24, 8, 46, 47, 90832, tzinfo=<UTC>)